In [ ]:
import csv
import pandas as pd
import os
import onnxruntime
import numpy as np
import z3
import gzip

In [ ]:
final = pd.DataFrame(columns=['bn','mo', 'pro', 'ar','at','mr','mt','mar','mat','nr','nt','vr','vt','pr','pt','cr','ct','dr','dt'])
ins = 'instances.csv'
result = 'results.csv'
benchmarks=['acasxu','collins_rul_cnn','mnist_fc','rl_benchmarks','tllverifybench','traffic_signs_recognition']

benchmarks_paths={'acasxu': './benchmarks/acasxu',
                  'collins_rul_cnn':'./benchmarks/collins_rul_cnn',
                  'mnist_fc':'./benchmarks/mnist_fc',
                  'rl_benchmarks':'./benchmarks/rl_benchmarks', 
                  'tllverifybench':'./benchmarks/tllverifybench',
                  'traffic_signs_recognition':'./benchmarks/traffic_signs_recognition'}

In [ ]:
def read_benchmark_csv_add_to_final(benchmark_name, benchmark_path, prefix):

    with open(benchmark_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            model =os.path.join(prefix,row[0])
            prop = os.path.join(prefix,row[1])
            time_out = row[2]
            final.loc[len(final)] = { 'bn':benchmark_name, 'mo':model,'pro': prop}

def all_benchmarks():
    for i in benchmarks_paths:
        path = os.path.join(benchmarks_paths[i], ins)
        prefix = './benchmarks/'+i
        # print(i,path,prefix)
        read_benchmark_csv_add_to_final(i,path,prefix)
    

all_benchmarks()
print(final.to_string())    

In [ ]:
def read_and_write_verification_result(result_csv_path,key_1,key_2):
    '''Read each verifier result csv file and add to the dataframe final '''
    with open(result_csv_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            benchmark_name = row[0]
            if benchmark_name in benchmarks:
                # print(benchmark_name)
                condition_model = row[1]
                # print(condition_model)
                condition_prop = row[2]
                # print(condition_prop)
                if row[4]=='sat' or row[4]=='unsat':
                    ver_res = row[4]
                else:
                    ver_res = 'unknown'
                ver_time = row[5]
                final.loc[(final['bn']==benchmark_name )& (final['mo']==condition_model) & (final['pro']==condition_prop), key_1] = ver_res
                final.loc[(final['bn']==benchmark_name )& (final['mo']==condition_model)  & (final['pro']==condition_prop), key_2] = ver_time

ce_results = pd.DataFrame(columns=['bn','mo', 'pro', 'ca','cm','cma','cn','cv','cp','cc','cd'])
results_path = {'abcrown':'./verification_results/abcrown',
                'cgdtest':'./verification_results/cgdtest',
                'debona':'./verification_results/marabou',
                'marabou':'./verification_results/marabou',
                'mn_bab':'./verification_results/mn_bab',
                'nnenum':'./verification_results/nnenum',
                'peregrinn':'./verification_results/peregrinn',
                'verinet':'./verification_results/verinet'
                }


def add_results():
    path_1 = './verification_results/abcrown/results.csv'
    path_2 = './verification_results/cgdtest/results.csv'
    path_3 = './verification_results/marabou/results.csv'
    path_4 = './verification_results/debona/results.csv'
    path_5 = './verification_results/mn_bab/results.csv'
    path_6 = './verification_results/peregrinn/results.csv'
    path_7 = './verification_results/verinet/results.csv'
    path_8 = './verification_results/nnenum/results.csv'
    read_and_write_verification_result(path_1,'ar','at')
    read_and_write_verification_result(path_2,'cr','ct')
    read_and_write_verification_result(path_3,'mar','mat')
    read_and_write_verification_result(path_4,'dr','dt')
    read_and_write_verification_result(path_5,'mr','mt')
    read_and_write_verification_result(path_6,'pr','pt')
    read_and_write_verification_result(path_7,'vr','vt')
    read_and_write_verification_result(path_8,'nr','nt')
    print(final.to_string())
add_results()

In [ ]:
# run this cell only when check the stablization

# precentage : 1, 0.8, 0.6, 0.4,
# precentage = 1
# random_n = int(711 * precentage)
# final_2 = final.sample(n=random_n)
# final = final_2

In [ ]:
def read_ce_file(ce_path):

    if ce_path.endswith('.gz'):
        with gzip.open(ce_path, 'rb') as f:
            content = f.read().decode('utf-8')
    else:
        with open(ce_path, 'r', encoding='utf-8') as f:
            content = f.read()

    content = content.replace('\n', ' ').strip()

    return content


# check counterexamples



map_2 = {'abcrown':'ca','cgdtest':'cc','debona':'cd','marabou':'cma','mn_bab':'cm','nnenum':'cn','peregrinn':'cp','verinet':'cv'}

def read_ce(filename: str):
    with open(filename, 'r') as f:
        content = f.readlines()
    input = list()
    for line in content:
        if "X" in line: 
            line = line.replace("(", "").replace(")","").replace(",", "")
            value = line.split()[-1]
            input.append(float(value))
    f.close()
    return input


def inference_network(model_path:str, ce_input: list):
    input_data = np.array(ce_input,dtype=np.float32)
    sess = onnxruntime.InferenceSession(model_path)
    input_name = sess.get_inputs()[0].name
    input_shape = sess.get_inputs()[0].shape
    for i in range(len(input_shape)):
            if not isinstance(input_shape[i],int):
                input_shape[i]=1
    input_data = input_data.reshape(input_shape)
    if input_data.shape != tuple(input_shape):
        raise ValueError(f"Input data shape does not match the model. Expected: {input_shape}, Got: {input_data.shape}")
    outputs = sess.run(None, {input_name: input_data})
    output = outputs[0]
    return output


def read_properties(prop_path:str):
    with open(prop_path, 'r') as f:
        content = f.readlines()
    y_properties = " "
    add_output_flag = False
    for line in content:
        if "Y" in line and "declare" in line: 
            y_properties += line
        elif "Output constraints" in line or "unsafe" in line or "coc" in line or "Unsafe" in line:
            add_output_flag = True
        elif add_output_flag == True:
            y_properties += line
    f.close()
    return y_properties

def reason(y_properties, output):
    # collectY_network_list = ["(assert (= Y_{} {}))".format(i, output[i]) for i in range(len(output))]
    collectY_network_list = list()
    for j in range(len(output)):
        single_value = output[j] # iteration over array
        if "e" in str(single_value):
            r_value = str(single_value).split("e")[0]
            res = int(str(single_value).split("e")[1])
            collectY_network_list.append("(assert (= Y_{} (* {} (^ 10 {}))))".format(j, float(r_value), res))
        else:
            collectY_network_list.append("(assert (= Y_{} {}))".format(j, single_value))
    collectY_network_output_constraint = "".join(collectY_network_list)
    combine = y_properties + collectY_network_output_constraint
    combine = combine.replace("\n", "").encode(encoding='utf-8')
    solver = z3.Solver()
    solver.reset()
    try:   
        SMT_properties = z3.parse_smt2_string(combine)
        solver.add(SMT_properties)
    except z3.Z3Exception:
        print(f"Error {combine}")
    return solver.check()

base = '/home/feng/vnncomp2022_benchmarks'


def add_to_ce_results(benchmark_name,model,prop):
    ce_results.loc[len(ce_results)] = {'bn':benchmark_name,'mo':model,'pro':prop}


def find_counterexample_paths():
    for i, r in final.iterrows():
        benchmark_name = r['bn']
        model = r['mo']
        prop = r['pro']
        model_name = model.split('/')[-1][:-5]
        prop_name = prop.split('/')[-1][:-7]
        ce_name = model_name+'_'+prop_name+'.counterexample'
        # ar = 
        if r['ar'] == 'sat':
            add_to_ce_results(benchmark_name,model,prop)
        # if r['bn']=='traffic_signs_recognition':
        #     continue
        if r['ar'] == 'sat':
            check_counterexample(benchmark_name,ce_name,model,prop,'abcrown')
        if r['mr'] == 'sat':
            check_counterexample(benchmark_name,ce_name,model,prop,'mn_bab')
        if r['mar'] == 'sat':
            check_counterexample(benchmark_name,ce_name,model,prop,'marabou')
        if r['nr'] == 'sat':
            check_counterexample(benchmark_name,ce_name,model,prop,'nnenum')
        if r['vr'] == 'sat':
            check_counterexample(benchmark_name,ce_name,model,prop,'verinet')
        if r['pr'] == 'sat':
            check_counterexample(benchmark_name,ce_name,model,prop,'peregrinn')
        if r['cr'] == 'sat':
            check_counterexample(benchmark_name,ce_name,model,prop,'cgdtest')
        if r['dr'] == 'sat':
            check_counterexample(benchmark_name,ce_name,model,prop,'debona')
        

def additional_check():
    temp = 0 
    tem = 0     
    for i, r in ce_results.iterrows():
        if r['bn']=='traffic_signs_recognition':
            temp +=1
            if temp in [14,21,27]:
                ce_results.at[i,'ca']='error'
            else:
                ce_results.at[i,'ca']='correct'
            tem +=1
            if tem ==27:
                ce_results.at[i,'cma']='error'
            if tem in [1,2,3,4,5,10,11,12,13,18,24,25,26,28,33,40,41,42]:
                ce_results.at[i,'cma']='correct'



def check_counterexample(benchmark_name, ce_name, model, prop, c):
    ce_path = os.path.join(results_path[c],benchmark_name,ce_name)
    if os.path.exists(ce_path):
        model_path = base+model[1:]
        print(model_path)
        prop_path = base+prop[1:]
        input = read_ce(ce_path)
        if len(input) <2:
            ce_results.loc[(ce_results['bn']==benchmark_name)& (ce_results['mo']==model)& (ce_results['pro']==prop), map_2[c]] = 'error'
        else:
            network_output = inference_network(model_path,input)
            y_properties = read_properties(prop_path)
            try:
                check_result = reason(y_properties,network_output)
            
                if check_result == z3.unsat:
                    ce_results.loc[(ce_results['bn']==benchmark_name)& (ce_results['mo']==model)& (ce_results['pro']==prop), map_2[c]] = "error"
                    # ce_results.loc[len(ce_results)] = {'bn':benchmark_name,'mo':model,'pro':prop,map_2[c]:'error'}
                elif check_result == z3.sat:
                    ce_results.loc[(ce_results['bn']==benchmark_name)& (ce_results['mo']==model)& (ce_results['pro']==prop), map_2[c]] = "correct"
                    
                    # ce_results.loc[len(ce_results)] = {'bn':benchmark_name,'mo':model,'pro':prop,map_2[c]:'correct'}
                else:
                    ce_results.loc[(ce_results['bn']==benchmark_name)& (ce_results['mo']==model)& (ce_results['pro']==prop), map_2[c]] = "???"

                    # ce_results.loc[len(ce_results)] = {'bn':benchmark_name,'mo':model,'pro':prop,map_2[c]:'??'}
            except:
                ce_results.loc[(ce_results['bn']==benchmark_name)& (ce_results['mo']==model)& (ce_results['pro']==prop), map_2[c]] = "?"

                # ce_results.loc[len(ce_results)] = {'bn':benchmark_name,'mo':model,'pro':prop,map_2[c]:'?'}
    additional_check()        


        


def report_ce(data: pd.DataFrame, toolsname:str):
    # conservative_report_pass(data, toolsname)
    # agressive_report_pass(data, toolsname)
    # agressive_report_nonrobust(data, toolsname)
    # conservative_report_nonrobust(data, toolsname)
    data = data[data["result"] =="sat"]
    fp_input = 0
    for index, row in data.iterrows():
        ce_path = os.path.splitext(row["network_path"])[0] \
            +"_" + os.path.splitext(os.path.basename(row["property_path"]))[0] \
            + ".counterexample"
        ce_path = ce_path.replace("benchmarks", "survey_data/{}".format(toolsname), 1).replace("onnx/", "")
        libproperty = read_properties(row["property_path"])
        input = read_ce(ce_path)
        if len(input) ==0:
            continue
        output = inference_network(row["network_path"], input)
        check_N = reason(libproperty, output[0])
        if check_N == z3.unsat:
            fp_input += 1
            with open(f"{toolsname}_Verification_input_fp.txt", "a") as fileA:
                # fileA.write("\n" + row.to_json())
                fileA.write(f"property_path:\t {ce_path} \n network_outputs:\t {output} \n")
        if check_N == z3.sat:
            lines = [f"{toolsname}", f"{row['network_path']}", f"{row['property_path']}", f"{ce_path}"]
            with open(f"{data['application'].iloc[1]}_real_ce.csv", mode= "a") as fileB:
                writer = csv.writer(fileB)
                writer.writerow(lines)
    print(f"input fp: {fp_input} using {toolsname} for {data['application'].iloc[1]} \n")
    return
find_counterexample_paths()

In [ ]:
ce_results.fillna('unknown', inplace=True)
print(ce_results.to_string())

# Ground truth

In [ ]:
violate_ground_truth = pd.DataFrame(columns=['bn','mo', 'pro', 'gt'])
for index, row in ce_results.iterrows():
    if row['ca']=='correct' or row['cm']=='correct' or row['cma']=='correct' or row['cn']=='correct' or row['cv']=='correct' or row['cp']=='correct' or row['cc']=='correct' or row['cd']=='correct':
        violate_ground_truth.loc[len(violate_ground_truth)] = {'bn':row['bn'],'mo':row['mo'],'pro':row['pro'],'gt':'sat'}
print(violate_ground_truth.to_string())

desirable_ground_truth = pd.DataFrame(columns=['bn','mo', 'pro', 'gt'])
for index, row in final.iterrows():
    if row['ar']=='sat' or row['mr']=='sat' or row['mar']=='sat' or row['nr']=='sat' or row['vr']=='sat' or row['pr']=='sat' or row['cr']=='sat' or row['dr']=='sat':
        continue
    if row['ar']=='unsat' or row['mr']=='unsat' or row['mar']=='unsat' or row['nr']=='unsat' or row['vr']=='unsat' or row['pr']=='unsat' or row['cr']=='unsat' or row['dr']=='unsat':
        desirable_ground_truth.loc[len(desirable_ground_truth)] = {'bn':row['bn'],'mo':row['mo'],'pro':row['pro'],'gt':'unsat'}

print(desirable_ground_truth.to_string())

# Soundness and completeness ratio

In [ ]:

count_1 ={'abcrown':0,'mnbab':0, 'marabou':0, 'verinet':0,'nnenum':0,'peregrinn':0,'debona':0,'cgdtest':0}
count_2 ={'abcrown':0,'mnbab':0, 'marabou':0, 'verinet':0,'nnenum':0,'peregrinn':0,'debona':0,'cgdtest':0}
count_3 ={'abcrown':0,'mnbab':0, 'marabou':0, 'verinet':0,'nnenum':0,'peregrinn':0,'debona':0,'cgdtest':0}
count_4 ={'abcrown':0,'mnbab':0, 'marabou':0, 'verinet':0,'nnenum':0,'peregrinn':0,'debona':0,'cgdtest':0}
count_time_1 ={'abcrown':0,'mnbab':0, 'marabou':0, 'verinet':0,'nnenum':0,'peregrinn':0,'debona':0,'cgdtest':0}
count_time_2 ={'abcrown':0,'mnbab':0, 'marabou':0, 'verinet':0,'nnenum':0,'peregrinn':0,'debona':0,'cgdtest':0}
count_time_3 ={'abcrown':0,'mnbab':0, 'marabou':0, 'verinet':0,'nnenum':0,'peregrinn':0,'debona':0,'cgdtest':0}
count_time_4 ={'abcrown':0,'mnbab':0, 'marabou':0, 'verinet':0,'nnenum':0,'peregrinn':0,'debona':0,'cgdtest':0}
for index, row in violate_ground_truth.iterrows():
    model = row['mo']
    prop = row['pro']
    # find the row in final based on model and prop
    value_final = final.loc[(final['mo']==model)& (final['pro']==prop)]
    if value_final['ar'].iloc[0]=='unsat' or value_final['ar'].iloc[0]=='unknown':
        count_2['abcrown']+=1
        count_time_2['abcrown']+=float(value_final['at'].iloc[0])
    if value_final['mr'].iloc[0]=='unsat' or value_final['mr'].iloc[0]=='unknown':
        count_2['mnbab']+=1
        count_time_2['mnbab']+=float(value_final['mt'].iloc[0])
    if value_final['mar'].iloc[0]=='unsat' or value_final['mar'].iloc[0]=='unknown':
        count_2['marabou']+=1
        count_time_2['marabou']+=float(value_final['mat'].iloc[0])
    if value_final['vr'].iloc[0]=='unsat' or value_final['vr'].iloc[0]=='unknown':
        count_2['verinet']+=1
        count_time_2['verinet']+=float(value_final['vt'].iloc[0])
    if value_final['nr'].iloc[0]=='unsat' or value_final['nr'].iloc[0]=='unknown':
        count_2['nnenum']+=1
        count_time_2['nnenum']+=float(value_final['nt'].iloc[0])
    if value_final['pr'].iloc[0]=='unsat' or value_final['pr'].iloc[0]=='unknown':
        count_2['peregrinn']+=1
        count_time_2['peregrinn']+=float(value_final['pt'].iloc[0])
    if value_final['cr'].iloc[0]=='unsat' or value_final['cr'].iloc[0]=='unknown':
        count_2['cgdtest']+=1
        count_time_2['cgdtest']+=float(value_final['ct'].iloc[0])
    if value_final['dr'].iloc[0]=='unsat' or value_final['dr'].iloc[0]=='unknown':
        count_2['debona']+=1
        count_time_2['debona']+=float(value_final['dt'].iloc[0])
    if value_final['ar'].iloc[0]=='sat':
        count_3['abcrown']+=1
        count_time_3['abcrown']+=float(value_final['at'].iloc[0])
    if value_final['mr'].iloc[0]=='sat':
        count_3['mnbab']+=1
        count_time_3['mnbab']+=float(value_final['mt'].iloc[0])
    if value_final['mar'].iloc[0]=='sat':
        count_3['marabou']+=1
        count_time_3['marabou']+=float(value_final['mat'].iloc[0])
    if value_final['vr'].iloc[0]=='sat':
        count_3['verinet']+=1
        count_time_3['verinet']+=float(value_final['vt'].iloc[0])
    if value_final['nr'].iloc[0]=='sat':
        count_3['nnenum']+=1
        count_time_3['nnenum']+=float(value_final['nt'].iloc[0])
    if value_final['pr'].iloc[0]=='sat':
        count_3['peregrinn']+=1
        count_time_3['peregrinn']+=float(value_final['pt'].iloc[0])
    if value_final['cr'].iloc[0]=='sat':
        count_3['cgdtest']+=1
        count_time_3['cgdtest']+=float(value_final['ct'].iloc[0])
    if value_final['dr'].iloc[0]=='sat':
        count_3['debona']+=1
        count_time_3['debona']+=float(value_final['dt'].iloc[0])

for index, row in desirable_ground_truth.iterrows():
    model = row['mo']
    prop = row['pro']
    value_final = final.loc[(final['mo']==model)& (final['pro']==prop)]
    if value_final['ar'].iloc[0]=='unsat' or value_final['ar'].iloc[0]=='unknown':
        count_1['abcrown']+=1
        count_time_1['abcrown']+=float(value_final['at'].iloc[0])
    if value_final['mr'].iloc[0]=='unsat' or value_final['mr'].iloc[0]=='unknown':
        count_1['mnbab']+=1
        count_time_1['mnbab']+=float(value_final['mt'].iloc[0])
    if value_final['mar'].iloc[0]=='unsat' or value_final['mar'].iloc[0]=='unknown':
        count_1['marabou']+=1
        count_time_1['marabou']+=float(value_final['mat'].iloc[0])
    if value_final['vr'].iloc[0]=='unsat' or value_final['vr'].iloc[0]=='unknown':
        count_1['verinet']+=1
        count_time_1['verinet']+=float(value_final['vt'].iloc[0])
    if value_final['nr'].iloc[0]=='unsat' or value_final['nr'].iloc[0]=='unknown':
        count_1['nnenum']+=1
        count_time_1['nnenum']+=float(value_final['nt'].iloc[0])
    if value_final['pr'].iloc[0]=='unsat' or value_final['pr'].iloc[0]=='unknown':
        count_1['peregrinn']+=1
        count_time_1['peregrinn']+=float(value_final['pt'].iloc[0])
    if value_final['cr'].iloc[0]=='unsat' or value_final['cr'].iloc[0]=='unknown':
        count_1['cgdtest']+=1
        count_time_1['cgdtest']+=float(value_final['ct'].iloc[0])
    if value_final['dr'].iloc[0]=='unsat' or value_final['dr'].iloc[0]=='unknown':
        count_1['debona']+=1
        count_time_1['debona']+=float(value_final['dt'].iloc[0])

    if value_final['ar'].iloc[0]=='sat' or value_final['ar'].iloc[0]=='unknown':
        count_4['abcrown']+=1
        count_time_4['abcrown']+=float(value_final['at'].iloc[0])
    if value_final['mr'].iloc[0]=='sat' or value_final['mr'].iloc[0]=='unknown':
        count_4['mnbab']+=1
        count_time_4['mnbab']+=float(value_final['mt'].iloc[0])
    if value_final['mar'].iloc[0]=='sat' or value_final['mar'].iloc[0]=='unknown':
        count_4['marabou']+=1
        count_time_4['marabou']+=float(value_final['mat'].iloc[0])
    if value_final['vr'].iloc[0]=='sat' or value_final['vr'].iloc[0]=='unknown':
        count_4['verinet']+=1
        count_time_4['verinet']+=float(value_final['vt'].iloc[0])
    if value_final['nr'].iloc[0]=='sat' or value_final['nr'].iloc[0]=='unknown':
        count_4['nnenum']+=1
        count_time_4['nnenum']+=float(value_final['nt'].iloc[0])
    if value_final['pr'].iloc[0]=='sat' or value_final['pr'].iloc[0]=='unknown':
        count_4['peregrinn']+=1
        count_time_4['peregrinn']+=float(value_final['pt'].iloc[0])
    if value_final['cr'].iloc[0]=='sat' or value_final['cr'].iloc[0]=='unknown':
        count_4['cgdtest']+=1
        count_time_4['cgdtest']+=float(value_final['ct'].iloc[0])
    if value_final['dr'].iloc[0]=='sat' or value_final['dr'].iloc[0]=='unknown':
        count_4['debona']+=1
        count_time_4['debona']+=float(value_final['dt'].iloc[0])

print(count_1,'\n',count_2,'\n',count_3,'\n',count_4,'\n')
print(count_time_1,'\n',count_time_2,'\n',count_time_3,'\n',count_time_4,'\n')

soundness={'abcrown':0,'mnbab':0, 'marabou':0, 'verinet':0,'nnenum':0,'peregrinn':0,'debona':0,'cgdtest':0}
completeness={'abcrown':0,'mnbab':0, 'marabou':0, 'verinet':0,'nnenum':0,'peregrinn':0,'debona':0,'cgdtest':0}
a_mean_sound_time={'abcrown':0,'mnbab':0, 'marabou':0, 'verinet':0,'nnenum':0,'peregrinn':0,'debona':0,'cgdtest':0}
a_mean_complete_time={'abcrown':0,'mnbab':0, 'marabou':0, 'verinet':0,'nnenum':0,'peregrinn':0,'debona':0,'cgdtest':0}
for i in count_1:
    soundness[i] = count_1[i]/(count_1[i]+count_2[i])
    completeness[i] = count_3[i]/(count_3[i]+ count_4[i])
    a_mean_sound_time[i] = (count_time_1[i]+count_time_2[i])/(count_1[i]+count_2[i])
    a_mean_complete_time[i] = (count_time_3[i]+count_time_4[i])/(count_3[i]+count_4[i])

print("soundness: ")
print(soundness)
print("completeness: ")
print(completeness)

print('\n')
print("mean sound time: ", a_mean_sound_time)
print("mean complete time: ", a_mean_complete_time)  
    

### Execute all cell above for the performance of the verifiers. Please note precentage in cell 5 should be 1.


### The following cells are used for the stablization evluation, change the precentage value to 0.8, 0.6 and 0.4 in cell 5. 


In [ ]:
# transfer to Dataframe type
soundness_df = pd.DataFrame(list(soundness.items()),columns = ['verifier','soundness'])
completeness_df = pd.DataFrame(list(completeness.items()),columns = ['verifier','completeness'])


# instore to csv file
soundness_df.to_csv('./results/soundness_A.csv', mode='a', header=False, index=False)
completeness_df.to_csv('./results/completeness_A.csv', mode='a',  header=False,index=False)

### For the stablization, the experiment data in the paper runs above cells 10 times. However, you can run as many time as you like. After running above cells, run the following cells.

In [ ]:
df = pd.read_csv('./results/completeness_A.csv')
grouped = df.groupby('verifier')['completeness']
means = grouped.mean()
print(means)
variances = grouped.var()
print(variances)

In [ ]:
df = pd.read_csv('./results/soundness_A.csv')
grouped = df.groupby('verifier')['soundness']
means = grouped.mean()
print(means)
variances = grouped.var()
print(variances)